(To do once per environment)
Clone the package from https://github.com/ClemenceK/bpifrance_deeptech_analysis


In [3]:
! pip install ..
# or if you prefer move to that project folder (where the setup.py is) in your terminal 
# and install from there using pip install .

Processing /Users/admin/code/ClemenceK/bpifrance_deeptech_analysis


  Created wheel for bpideep: filename=bpideep-1.0-py3-none-any.whl size=3213102 sha256=55368f24c984c7d71790b1b8a0c2a7d420c437216eeeee6cce22c5ef04c2ff88
  Stored in directory: /private/var/folders/63/jgrvs5g554j3k8z2rpggymy80000gn/T/pip-ephem-wheel-cache-b0edc1o2/wheels/3c/a8/6c/083ace2ed1bd0b270918b5638c589be45a698c36a794d9b15d
Successfully built bpideep


  Attempting uninstall: bpideep
    Found existing installation: bpideep 1.0
    Uninstalling bpideep-1.0:
      Successfully uninstalled bpideep-1.0
You should consider upgrading via the '/Users/admin/.pyenv/versions/NLP/bin/python -m pip install --upgrade pip' command.


Clone in the same place the package from: https://github.com/ClemenceK/deep4deep

In [5]:
# install the bpifrance_deeptech_analysis package from here…
! pip install ../../deep4deep/
# or if you prefer move to that project folder (where the setup.py is) in your terminal 
# and install from there using pip install .

Processing /Users/admin/code/ClemenceK/deep4deep
ERROR: Could not find a version that satisfies the requirement csv (from deep4deep)
ERROR: No matching distribution found for csv
You should consider upgrading via the '/Users/admin/.pyenv/versions/NLP/bin/python -m pip install --upgrade pip' command.


In [8]:
#this can be used to test if the import worked
from deep4deep.utils import simple_time_tracker

@simple_time_tracker
def test():
    print("It works if below is printed something like: test 0.0, or another figure")
    
test()

It works if below is printed something like: test 0.0, or another figure
test 0.0


## Building the base data from dealroom

Input must be of the form:
[TODO Cath] liste de noms d'entreprises ou d'id dealroom…

Then use functions:
[TODO Cath]

At this point the data should be like this:
[TODO Cath/check Flo]

## Scraping and adding the LinkedIn data

The first step is to generate scripts for webscraper and scrape LinkedIn -> live demo.

In [2]:
# Prior to calling the function "build employee_df", the csv containing the scrapped data should be included
# in a folder 'bpi_deep/scraping_data/companies_people/'
from bpideep.process_scraped_data import build_employee_df, process_employee_data
df_employees= process_employee_data(build_employee_df())

In [4]:
#Example of the content of the employee dataframe after processing
df_employees.head(5)

,employee_name,title,profile-href,linkedin_url,technical,founder,phd
0,Vincent Tournier,responsable ingénierie enzymatique,https://www.linkedin.com/in/vincenttournier1/,https://www.linkedin.com/company/carbios,0,0,0
1,Virginie Breillet,chef de projet,https://www.linkedin.com/in/virginie-breillet-...,https://www.linkedin.com/company/mastergrid,0,0,0
2,Ludivine Bonnet,responsable rh,https://www.linkedin.com/in/ludivine-bonnet-60...,https://www.linkedin.com/company/carbios,0,0,0
3,Justine Rayssac,molecular biology engineer intern,https://www.linkedin.com/in/justine-rayssac-75...,https://www.linkedin.com/company/gourmey,1,0,0
4,Henrique Silvério,quantum software engineer,https://www.linkedin.com/in/henrique-silv%C3%A...,https://www.linkedin.com/company/pasqal,0,0,0


At this point, generate scripts for profile scraping on LinkedIn and scrape

In [6]:
# Prior to calling the function "open_founder_profile_files", the csv containing the scrapped data from founders
# should be included in a folder 'bpi_deep/scraping_data/founders_files/'
from bpideep.process_scraped_data import open_founder_profile_files, inline_profile
from bpideep.process_scraped_data import build_founders_dataframe, generate_founders_features
df_founders_raw = open_founder_profile_files()

In [7]:
#The function "build_founders_dataframe" processes the raw df and returns a df with one line per founder
#The function "generate_founders_features" generates the new relevant features such as "founder_has_phd" etc..
df_founders = generate_founders_features(build_founders_dataframe(df_founders_raw))

/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/bpideep/process_scraped_data.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf[f'{field}_{i+1}'] = subdf.loc[i, field]
/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/bpideep/process_scraped_data.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf[f'{field}_{i+1}'] = subdf.loc[i, field]
/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/bpideep/process_scraped_data.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf[f'{field}_{i+1}'] = subdf.loc[i, field]
/home/florent/.pyenv/versions/3.8.5/envs/deep4deep/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [8]:
df_founders.head(5)

,profile-href,title,company,exp_description,title_2,company_2,exp_description_2,title_3,company_3,exp_description_3,...,type_4,amount_4,text_content_4,type_5,amount_5,text_content_5,founder_has_phd,founder_from_institute,founder_pat_pub,technical_founder
0,https://www.linkedin.com/in/eliott-raoult/,Intern in charge of the web marketing,LE COMPTOIR DES SAVONNIERS PARIS,Environment : \nRestructuration of the company...,Creation of an e-commerce website,Vamana Sunglasses,E-commerce website that was selling sunglasses...,Vice-president and treasurer,L'Instant Start-Up,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,https://www.linkedin.com/in/christophe-mille-5...,Lithium-Ion Battery - Pilot Line Manager,CEA,Pilot Line implementation: Dry & Clean Room Fa...,Battery Engineering,Apple,NaN,Battery Cell Development,BMW,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1
2,https://www.linkedin.com/in/nicolasmorinforest/,Co-Founder & CEO,GOURMEY,On a mission to bring delicious cultured meat ...,Advisor,Agriculture Cellulaire France,Cellular Agriculture France\n(Unpaid advisor),International Product Manager,"L'Oréal / Vichy, Europe’s #1 pharmacy brand",Active Cosmetics Division - Vichy Internationa...,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,https://www.linkedin.com/in/dekelpersi/,Board Observer,Codota,NaN,"Co Founder, Managing Partner",TPY Capital,Investing now from our second $100m venture fu...,Board Member,Signals Analytics,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,https://www.linkedin.com/in/pauline-de-breteuil/,Investor,Lifen,Smart and secure medical exchange\nhttps://www...,Investor,Epigene Labs,Genomic data-driven drug design for precision ...,Board member,Sym Optic,"Sym developed the first clip-on, interchangeab...",...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [10]:
# Finally, we merge founders to the full employee DF, update the feature ("technical"), and aggregate into companies
from bpideep.process_scraped_data import companies_technical_stats_with_founders_features, update_technical
df_employees_full = update_technical(df_employees, df_founders)
df_companies_stats_with_founders_features = companies_technical_stats_with_founders_features(df_employees_full)
df_companies_stats_with_founders_features.head(5)

,linkedin_url,technical,phd_found_linkedin,employee__linkedin_count,founder_from_institute,founder_has_phd,founder_pat_pub,technical_founder
0,https://www.linkedin.com/company/carbios,0.315789,0,38,0.0,1.0,0.0,1.0
1,https://www.linkedin.com/company/epigene-labs,0.250000,0,12,0.0,1.0,0.0,1.0
2,https://www.linkedin.com/company/gourmey,0.526316,4,19,2.0,1.0,0.0,2.0
3,https://www.linkedin.com/company/kraaft-co,0.142857,0,14,0.0,0.0,0.0,0.0
4,https://www.linkedin.com/company/mastergrid,0.183908,0,87,0.0,0.0,0.0,0.0


In [12]:
# Last optional step: merge the DF with new company features witht the dealroom df.
import pandas as pd
from bpideep.process_scraped_data import merge_initial_companies_with_founder
df_full = pd.read_csv('../bpideep/rawdata/data2020-12-03.csv')
final = merge_initial_companies_with_founder(df_full, df_companies_stats_with_founders_features)

At this point the data should be like this:
[TODO Flo/Cath]

## Checking the NaNs

It is always best to have real data, even approximated and filled manually. 
However, for some cells, the pipeline will be able to fill in the missing data, using the average or most frequent data in the training set. It is allowed to have empty or NaN cells for the following fields:
 
+ 
+ 
+ 

If you have empty cells in any other fields, the model will throw an error, so make sure to fill them.

## Using the main model to predict whether the company is deeptech

Then use the following functions:
[TODO Cath]

## Scraping and using text data

The input needs to have the following fields:
[TODO]

Then use the following functions:

## Using the NLP model to predict whether the company is deeptech

## Making the two predictions "vote" for final prediction